In [1]:
import numpy as np
from scipy.spatial import ConvexHull
v = np.array([[0.0, 0.0,0.0,0.0],[1.0, 1.0,1.0,1.0]])
U,S,V = np.linalg.svd(v)
epsilon = 0.000001



In [5]:
print (S)
dim_to_remove = np.argmax(np.abs(S) < epsilon)
new_pts = np.matmul(U,np.diag(S))
print (new_pts)
print (new_pts[:, :dim_to_remove])

[2. 0.]
[[ 0.  0.]
 [-2.  0.]]
[[ 0.]
 [-2.]]


In [6]:
def toHull(low_dim_pts_reduct):
    if low_dim_pts_reduct.shape[1] == 1:
        lb = np.min(low_dim_pts_reduct)
        ub = np.max(low_dim_pts_reduct)
        # x <= ub  -x >= -lb
        A = np.array([[1.0],[-1.0]])
        b = np.array([ub, -lb])
    else:
        # convex hull will get you Ax + b <= 0
        hull = ConvexHull(points=low_dim_pts_reduct)
        A = hull.equations[:,:-1]
        b = -hull.equations[:,-1]
    print (A,b)
    return A,b

In [7]:
low_dim_pts_reduct = new_pts[:, :dim_to_remove]
A,b = toHull(low_dim_pts_reduct)

[[ 1.]
 [-1.]] [0. 2.]


In [9]:
# complete to the full dimension
n_rows = A.shape[0]
n_cols = A.shape[1]

num_var_to_add = v.shape[1] - (dim_to_remove)

zero4 = np.zeros((n_rows,num_var_to_add))

rows_to_append = []
for idx in range(num_var_to_add):
    l = [0.0]*num_var_to_add
    l[idx] = 1.0
    rows_to_append.append(np.append(np.zeros((1,n_cols)), l))
    l[idx] = -1.0
    rows_to_append.append(np.append(np.zeros((1,n_cols)), l))

trueA = np.vstack([np.hstack([A, zero4])] + rows_to_append)
trueB = np.append(b, [0.0, 0.0]*num_var_to_add)
print (trueA)
print (trueB)

[[ 1.  0.  0.  0.]
 [-1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0. -1.]]
[0. 2. 0. 0. 0. 0. 0. 0.]


In [10]:
finalA = np.matmul(trueA, V)
finalA

array([[-0.5       , -0.5       , -0.5       , -0.5       ],
       [ 0.5       ,  0.5       ,  0.5       ,  0.5       ],
       [ 0.8660254 , -0.28867513, -0.28867513, -0.28867513],
       [-0.8660254 ,  0.28867513,  0.28867513,  0.28867513],
       [ 0.        , -0.57735027,  0.78867513, -0.21132487],
       [ 0.        ,  0.57735027, -0.78867513,  0.21132487],
       [ 0.        , -0.57735027, -0.21132487,  0.78867513],
       [ 0.        ,  0.57735027,  0.21132487, -0.78867513]])

In [11]:
0.57735027/0.21132487

2.7320507520009363

In [12]:
-0.78867513/0.21132487

-3.7320507046804288

In [13]:
a_ = 0.57735027; b_ = 0.78867513; c_ = 0.21132487;
(b_ - c_)/(a_)

0.9999999826794919

In [14]:
a_+c_-b_

1.0000000050247593e-08